## import json file

In [32]:
import json
file = "testBF.json"
#Read JSON data into the datastore variable
if file:
    with open(file, 'r') as f:
        packets = json.load(f)

In [37]:
len(packets)

2889

## total packets = total number of packets sent in the flow

## total bytes = total number of bytes sent in the flow

## Dataframe

In [38]:
list_test =[]

for i in range(0, len(packets)):
    ip_src = packets[i]['_source']['layers']['ip']['ip.src']
    ip_dst = packets[i]['_source']['layers']['ip']['ip.dst']
    srcport = packets[i]['_source']['layers']['tcp']['tcp.srcport']
    dstport = packets[i]['_source']['layers']['tcp']['tcp.dstport']
    flags= packets[i]['_source']['layers']['tcp']['tcp.flags_tree']
    flag_ack = flags['tcp.flags.ack']
    flag_syn = flags['tcp.flags.syn']
    flag_fin = flags['tcp.flags.fin']
    flag_psh = flags['tcp.flags.push']
    frame_len = int(packets[i]['_source']['layers']['frame']['frame.len'])
    tcp_len = int(packets[i]['_source']['layers']['tcp']['tcp.len'])
    time_relative = packets[i]['_source']['layers']['frame']['frame.time_relative']
    list_test.append({'frame_len': frame_len, 'ip_src': ip_src, 'ip_dst': ip_dst, 'srcport': srcport, \
                    'dstport': dstport, 'flag_ack':flag_ack, 'flag_syn':flag_syn, \
                    'flag_fin':flag_fin,'flag_psh':flag_psh , 'tcp_len':tcp_len, 'time_relative':time_relative})

print(len(list_test))

2889


In [46]:
import pandas as pd
df_test = pd.DataFrame(list_test)
df_test["time_relative"] = pd.to_numeric(df_test["time_relative"])
df_test["flag_ack"] = pd.get_dummies(df_test["flag_ack"])
df_test["flag_syn"] = pd.get_dummies(df_test["flag_syn"])
df_test["flag_fin"] = pd.get_dummies(df_test["flag_fin"])
df_test["flag_psh"] = pd.get_dummies(df_test["flag_psh"])
df_test.sort_values(by=['time_relative'])
df_test.head(50)

,frame_len,ip_src,ip_dst,srcport,dstport,flag_ack,flag_syn,flag_fin,flag_psh,tcp_len,time_relative
0,60,10.0.8.3,192.168.1.100,55108,22,1,0,1,1,0,0.000000
1,60,192.168.1.100,10.0.8.3,22,55108,0,0,1,1,0,0.004275
2,52,10.0.8.3,192.168.1.100,55108,22,0,1,1,1,0,0.004358
3,74,10.0.8.3,192.168.1.100,55108,22,0,1,1,0,22,0.004829
4,52,192.168.1.100,10.0.8.3,22,55108,0,1,1,1,0,0.008154
5,93,192.168.1.100,10.0.8.3,22,55108,0,1,1,0,41,0.017855
6,52,10.0.8.3,192.168.1.100,55108,22,0,1,1,1,0,0.017893
7,996,10.0.8.3,192.168.1.100,55108,22,0,1,1,0,944,0.018693
8,1132,192.168.1.100,10.0.8.3,22,55108,0,1,1,0,1080,0.021909
9,52,10.0.8.3,192.168.1.100,55108,22,0,1,1,1,0,0.021969


In [170]:
df = df_test.groupby(['ip_src' , 'ip_dst', 'dstport' , 'srcport']).agg(
    max_time=pd.NamedAgg(column='time_relative', aggfunc=max,), 
    min_time=pd.NamedAgg(column='time_relative', aggfunc=min), 
    total_packets=pd.NamedAgg(column='tcp_len', aggfunc=sum), 
    tot_bytes=pd.NamedAgg(column='frame_len', aggfunc=sum),
    ack = pd.NamedAgg(column='flag_ack', aggfunc=sum),
    syn = pd.NamedAgg(column='flag_syn', aggfunc=sum), 
    psh = pd.NamedAgg(column='flag_psh', aggfunc=sum),
    fin = pd.NamedAgg(column='flag_fin', aggfunc=sum))

df["duration"] = df["max_time"] - df["min_time"]

In [ ]:
df.loc[df['ack'] != 0 , 'ack'] = 1
df.loc[df['syn'] != 0 , 'syn'] = 1
df.loc[df['psh'] != 0 , 'psh'] = 1
df.loc[df['fin'] != 0 , 'fin'] = 1

In [143]:
df.head()

max_time  min_time  total_packets  \
ip_src   ip_dst        dstport srcport                                       
10.0.8.3 192.168.1.100 22      55108     0.133994  0.000000           1202   
                               55110    12.647232  0.372763           1654   
                               55112    12.658387  0.372743           1654   
                               55116    12.616369  0.373144           1654   
                               55118    12.619176  0.373139           1654   

                                        tot_bytes  ack  syn  psh  fin  \
ip_src   ip_dst        dstport srcport                                  
10.0.8.3 192.168.1.100 22      55108         1990    1    1    1    1   
                               55110         2650    1    1    1    1   
                               55112         2650    1    1    1    1   
                               55116         2650    1    1    1    1   
                               55118         2650    1    1    1    1   

                                         duration  
ip_src   ip_dst        dstport srcport             
10.0.8.3 192.168.1.100 22      55108     0.133994  
                               55110    12.274469  
                               55112    12.285644  
                               55116    12.243225  
                               55118    12.246037

In [ ]:
df.drop(['max_time', 'min_time'], axis=1, inplace=False)

In [150]:
df.reset_index(inplace=True) 
df.head()

,ip_src,ip_dst,dstport,srcport,total_packets,tot_bytes,ack,syn,psh,fin,duration
0,10.0.8.3,192.168.1.100,22,55108,1202,1990,1,1,1,1,0.133994
1,10.0.8.3,192.168.1.100,22,55110,1654,2650,1,1,1,1,12.274469
2,10.0.8.3,192.168.1.100,22,55112,1654,2650,1,1,1,1,12.285644
3,10.0.8.3,192.168.1.100,22,55116,1654,2650,1,1,1,1,12.243225
4,10.0.8.3,192.168.1.100,22,55118,1654,2650,1,1,1,1,12.246037


In [160]:
df['Label'] = 'ssh-bruteforce'

In [161]:
df.head()

,ip_src,ip_dst,dstport,srcport,total_packets,tot_bytes,ack,syn,psh,fin,duration,Label
0,10.0.8.3,192.168.1.100,22,55108,1202,1990,1,1,1,1,0.133994,ssh-bruteforce
1,10.0.8.3,192.168.1.100,22,55110,1654,2650,1,1,1,1,12.274469,ssh-bruteforce
2,10.0.8.3,192.168.1.100,22,55112,1654,2650,1,1,1,1,12.285644,ssh-bruteforce
3,10.0.8.3,192.168.1.100,22,55116,1654,2650,1,1,1,1,12.243225,ssh-bruteforce
4,10.0.8.3,192.168.1.100,22,55118,1654,2650,1,1,1,1,12.246037,ssh-bruteforce


In [167]:
df.shape

(152, 12)